<a href="https://colab.research.google.com/github/Huwa-hank/114-1PL/blob/main/HW1_%E7%94%9F%E6%B4%BB%E6%B6%88%E8%B2%BB%E7%B4%80%E9%8C%84%E8%A1%A8_Gradio%E7%89%88%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。

AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。

Sheet 欄位：date, category, item, amount, payer

In [1]:
!pip install gspread oauth2client gspread_dataframe

Week2 原代碼

In [3]:
import gspread
from google.auth import default
import pandas as pd
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.colab import auth
auth.authenticate_user()

from google.auth.transport.requests import Request
from google.auth import default

import datetime

# 🔑 如果用 Colab OAuth
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟 Google Sheet（請改成你的表單名稱）
sheet = gc.open("日常財務紀錄表").sheet1

# 初始化欄位
columns = ["日期", "項目", "分類", "金額"]

def date_enter():
    while True:
        try:
            Y = int(input("請輸入年份 (例如 2025): "))
            M = int(input("請輸入月份 (1-12): "))
            D = int(input("請輸入日期 (1-31): "))

            # 嘗試建立日期物件，檢查是否有效
            date_ob = datetime.date(Y, M, D)
            return date_ob.strftime("%Y-%m-%d")

        except ValueError:
            print("⚠️ 日期輸入錯誤，請重新輸入！")

def month_enter():
    while True:
        try:
            Y = int(input("請輸入年份 (例如 2025): "))
            M = int(input("請輸入月份 (1-12): "))

            if 1 <= M <= 12:
                date_ob = f"{Y}-{M:02d}"  # 格式化成 YYYY-MM
                return date_ob
            else:
                print("月份錯誤")

        except ValueError:
            print("⚠️ 日期輸入錯誤，請重新輸入！")

def read_rec():
    """從 Google Sheet 載入紀錄"""
    df = get_as_dataframe(sheet, evaluate_formulas=True, header=0)
    df = df.dropna(how="all")  # 移除空列
    if df.empty:
        df = pd.DataFrame(columns=columns)
    return df

def write_rec(df):
    """把 DataFrame 寫回 Google Sheet"""
    sheet.clear()
    set_with_dataframe(sheet, df)

def add_rec():
    df = read_rec()
    date = date_enter()
    cate = input("輸入項目: ")
    sort = input("輸入分類: ")
    mounth = float(input("輸入金額: "))
    new_row = pd.DataFrame([[date, cate, sort, mounth]], columns=columns)
    df = pd.concat([df, new_row], ignore_index=True)
    write_rec(df)
    print("✅ 已新增紀錄！")

def check_rec():
    df = read_rec()
    if df.empty:
        print("⚠️ 目前沒有紀錄")
    else:
        print(df)

def month_totle():
    df = read_rec()
    month = month_enter()
    month_data = df[df["日期"].str.startswith(month)]
    total = month_data["金額"].sum()
    print(f"💰 {month} 總額: {total}")

def sort_totle():
    df = read_rec()
    month = month_enter()
    month_data = df[df["日期"].str.startswith(month)]
    if month_data.empty:
        print("⚠️ 該月份沒有紀錄")
    else:
        print("📊 分類小計：")
        print(month_data.groupby("分類")["金額"].sum())

# 🔹 主選單
def demo():
    while True:
        print("\n=== 消費紀錄系統 ===")
        print("1. 新增紀錄")
        print("2. 查看紀錄")
        print("3. 計算月總額")
        print("4. 計算分類小計")
        print("0. 離開")

        choice = input("請輸入選項: ")

        if choice == "1":
            add_rec()
        elif choice == "2":
            check_rec()
        elif choice == "3":
            month_totle()
        elif choice == "4":
            sort_totle()
        elif choice == "0":
            print("👋 程式結束")
            break
        else:
            print("⚠️ 無效選項，請重新輸入")

# 🚀 執行
demo()



=== 消費紀錄系統 ===
1. 新增紀錄
2. 查看紀錄
3. 計算月總額
4. 計算分類小計
0. 離開
請輸入選項: 0
👋 程式結束


Chatgpt生成的Gardio示範


In [4]:
import gradio as gr
import pandas as pd
import os

FILE_NAME = "records.csv"

if not os.path.exists(FILE_NAME):
    df = pd.DataFrame(columns=["日期", "品項", "金額", "分類"])
    df.to_csv(FILE_NAME, index=False)

def add_record(日期, 品項, 金額, 分類):
    df = pd.read_csv(FILE_NAME)

    new_record = {"日期": 日期, "品項": 品項, "金額": 金額, "分類": 分類}
    df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)

    df.to_csv(FILE_NAME, index=False)

    # 回傳 (最新 5 筆紀錄, 清空日期, 清空品項, 清空金額, 清空分類)
    return df.tail(5), "", "", None, None

with gr.Blocks() as demo:
    gr.Markdown("## 📝 消費紀錄表 - 新增紀錄")

    日期 = gr.Textbox(label="日期 (YYYY-MM-DD)")
    品項 = gr.Textbox(label="品項")
    金額 = gr.Number(label="金額")
    分類 = gr.Dropdown(["餐飲", "交通", "娛樂", "購物", "其他"], label="分類")

    submit_btn = gr.Button("新增紀錄")
    output = gr.Dataframe(headers=["日期", "品項", "金額", "分類"], interactive=False)

    submit_btn.click(
        add_record,
        inputs=[日期, 品項, 金額, 分類],
        outputs=[output, 日期, 品項, 金額, 分類]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b169240e513703a6b4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


和Chatgpt調整並整合後的程式碼

In [10]:
import gradio as gr
import gspread
import pandas as pd
import datetime

from google.auth import default
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.colab import auth

# Colab OAuth 登入（你之前用過）
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟 Google Sheet（請確認名稱正確）
sheet = gc.open("日常財務紀錄表").sheet1

# 欄位定義（跟試算表欄位一致）
COLUMNS = ["日期", "品項", "分類", "金額"]

# 讀取 Google Sheet -> DataFrame
def read_rec():
    df = get_as_dataframe(sheet, evaluate_formulas=True, header=0)
    df = df.dropna(how="all")
    if df.empty:
        df = pd.DataFrame(columns=COLUMNS)
    # 若試算表的欄位名稱有小差異，可嘗試調整或確保一致
    return df.reset_index(drop=True)

# 寫回 Google Sheet
def write_rec(df):
    sheet.clear()
    set_with_dataframe(sheet, df)

# 小工具：驗證 YYYY-MM-DD
def is_valid_date(s):
    try:
        datetime.date.fromisoformat(s)
        return True
    except Exception:
        return False

# 新增紀錄（給 Gradio 用）
def add_rec(date, item, category, amount):
    # 驗證日期
    if not date or not is_valid_date(date):
        df_current = read_rec()
        return "⚠️ 日期格式錯誤（請輸入 YYYY-MM-DD）", df_current.tail(10), date, item, amount, category

    # 驗證金額
    try:
        amount_val = float(amount)
    except Exception:
        df_current = read_rec()
        return "⚠️ 金額格式錯誤（請輸入數字）", df_current.tail(10), date, item, amount, category

    # 讀資料、加入新列、寫回
    df = read_rec()
    new_row = pd.DataFrame([[date, item, category, amount_val]], columns=COLUMNS)
    df = pd.concat([df, new_row], ignore_index=True)
    write_rec(df)

    # 回傳：訊息、最新紀錄（顯示最後 10 筆）、並清空輸入框
    return "✅ 已新增紀錄！", df.tail(10), "", "", None, None

# 建 Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📝 消費紀錄表 - 新增紀錄")

    # 訊息顯示（用來回傳成功或錯誤訊息）
    msg = gr.Textbox(label="系統訊息", interactive=False)

    date = gr.Textbox(label="日期 (YYYY-MM-DD)")
    item = gr.Textbox(label="品項")
    amount = gr.Number(label="金額")
    category = gr.Dropdown(["餐飲", "交通", "娛樂", "購物", "其他"], label="分類")

    submit_btn = gr.Button("新增紀錄")
    output = gr.Dataframe(headers=COLUMNS, interactive=False)

    # 注意 outputs 的順序要和 add_rec 回傳值順序一樣
    submit_btn.click(
        add_rec,
        inputs=[date, item, category, amount],
        outputs=[msg, output, date, item, amount, category]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63b11b690c02f628fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Chatgpt生成切換Gardio介面示範

In [25]:
import gradio as gr

def 切到新增():
    return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

def 切到查看():
    return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)

def 回首頁():
    return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)

with gr.Blocks() as demo:
    # 首頁
    with gr.Column(visible=True) as home:
        gr.Markdown("# 🏠 主選單")
        to_add = gr.Button("📝 新增紀錄")
        to_view = gr.Button("📊 查看紀錄")

    # 新增頁面
    with gr.Column(visible=False) as add_page:
        gr.Markdown("## 📝 新增紀錄")
        back1 = gr.Button("⬅️ 回首頁")

    # 查看頁面
    with gr.Column(visible=False) as view_page:
        gr.Markdown("## 📊 查看紀錄")
        back2 = gr.Button("⬅️ 回首頁")

    # 事件控制
    to_add.click(切到新增, None, [add_page, view_page, home])
    to_view.click(切到查看, None, [add_page, view_page, home])
    back1.click(回首頁, None, [add_page, view_page, home])
    back2.click(回首頁, None, [add_page, view_page, home])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c4f19dac19534440ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


最後整合案(介面整合)

In [27]:
import gradio as gr
import gspread
import os
import pandas as pd
import datetime

from google.auth import default
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.colab import auth
from google.auth.transport.requests import Request

# 🔑 如果用 Colab OAuth
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟 Google Sheet（請改成你的表單名稱）
sheet = gc.open("日常財務紀錄表").sheet1

# 初始化欄位
columns = ["日期", "項目", "分類", "金額"]

# 小工具：驗證 YYYY-MM-DD
def is_valid_date(s):
    try:
        datetime.date.fromisoformat(s)
        return True
    except Exception:
        return False

# 讀取紀錄
def read_rec():
    """從 Google Sheet 載入紀錄"""
    df = get_as_dataframe(sheet, evaluate_formulas=True, header=0)
    df = df.dropna(how="all")  # 移除空列
    if df.empty:
        df = pd.DataFrame(columns=columns)
    return df

# 寫入紀錄
def write_rec(df):
    """把 DataFrame 寫回 Google Sheet"""
    sheet.clear()
    set_with_dataframe(sheet, df)

# 新增紀錄
def add_rec(date, cate, sort, amount):

# 驗證日期
    if not date or not is_valid_date(date):
        df_current = read_rec()
        return "⚠️ 日期格式錯誤（請輸入 YYYY-MM-DD）", df_current.tail(10), date, cate, sort, amount

# 驗證金額
    try:
        amount_val = float(amount)
    except Exception:
        df_current = read_rec()
        return "⚠️ 金額格式錯誤（請輸入數字）", df_current.tail(10), date, cate, sort, amount

    df = read_rec()
    new_row = pd.DataFrame([[date, cate, sort, amount]], columns=columns)
    df = pd.concat([df, new_row], ignore_index=True)
    write_rec(df)

    return "✅ 已新增紀錄！", df.tail(5)," "," ",None,None
# 檢查紀錄
def check_rec():
    data = sheet.get_all_records()
    if not data:
        return pd.DataFrame(), None
    df = pd.DataFrame(data)
    csv_path = "/tmp/消費紀錄.csv"
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    return df, csv_path

# 月總計
def month_totle():
    df = read_rec()
    month = month_enter()
    month_data = df[df["日期"].str.startswith(month)]
    total = month_data["金額"].sum()
    print(f"💰 {month} 總額: {total}")

# 分類總計
def sort_totle():
    df = read_rec()
    month = month_enter()
    month_data = df[df["日期"].str.startswith(month)]
    if month_data.empty:
        print("⚠️ 該月份沒有紀錄")
    else:
        print("📊 分類小計：")
        print(month_data.groupby("分類")["金額"].sum())

# Gradio 介面
with gr.Blocks() as demo:
        with gr.Tabs():
            # 分頁 1️⃣ 新增紀錄
            with gr.TabItem("新增紀錄"):
                gr.Markdown("## 📝 消費紀錄表 - 新增紀錄")

                date = gr.Textbox(label="日期 (YYYY-MM-DD)")
                cate = gr.Textbox(label="品項")
                amount = gr.Number(label="金額")
                sort = gr.Dropdown(["餐飲", "交通", "娛樂", "購物", "其他"], label="分類")

                submit_btn = gr.Button("新增紀錄")
                msg = gr.Textbox(label="系統訊息", interactive=False)
                output = gr.Dataframe(headers=["日期", "品項", "分類", "金額"], interactive=False)

                submit_btn.click(
                    add_rec,
                    inputs=[date, cate, sort, amount],
                    outputs=[msg,output,date, cate, sort, amount]
                )
            # 分頁 2️⃣ 查看紀錄
        with gr.TabItem("查看紀錄"):
            show_btn = gr.Button("📊 更新並顯示紀錄")
            table = gr.DataFrame(label="目前紀錄", interactive=False)
            download = gr.File(label="下載 CSV")

            show_btn.click(
                check_rec,
                inputs=[],
                outputs=[table, download]
            )


demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://95b91d92cdf987ab5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
